In [22]:
import pandas as pd
import numpy as np
import os
import joblib
import psutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [23]:
# Definir la ruta base del dataset procesado
RUTA_BASE = "C:/Users/oscar/Desktop/Henry/Proyectos/INDIVIDUAL 1/data_set"
ruta_procesada = os.path.join(RUTA_BASE, "movies_dataset_processed.parquet")
#Cargar el dataset procesado
movies_df = pd.read_parquet(ruta_procesada)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28926 entries, 0 to 28925
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  2164 non-null   object        
 1   genres                 28926 non-null  object        
 2   id                     28926 non-null  int64         
 3   original_language      28926 non-null  object        
 4   overview               28926 non-null  object        
 5   popularity             28926 non-null  float64       
 6   production_companies   28622 non-null  object        
 7   production_countries   28923 non-null  object        
 8   release_date           28926 non-null  datetime64[ns]
 9   revenue                28926 non-null  float64       
 10  runtime                28796 non-null  float64       
 11  spoken_languages       28908 non-null  object        
 12  tagline                14246 non-null  object        
 13  t

In [24]:
# Vectorización del texto con TF-IDF
# Se extraen características a partir del texto de la columna 'predictor'
vectorizer = TfidfVectorizer(min_df=4, max_df=0.85, ngram_range=(1, 2), max_features=40000, dtype=np.float32)
tfidf_matrix = vectorizer.fit_transform(movies_df['predictor'])

In [25]:
# Reducción de dimensionalidad con SVD (Latent Semantic Analysis)
# Se reduce la dimensión de la matriz TF-IDF para optimizar rendimiento y reducir ruido
svd = TruncatedSVD(n_components=400, random_state=42)
matriz_reducida = svd.fit_transform(tfidf_matrix)

In [26]:
# Función para recomendar películas basada en similitud de coseno
def recomendar_peliculas(titulo, num_recomendaciones=5):
    """ Retorna una lista de películas similares en base al título dado. """
    titulo = titulo.lower()
    idx = movies_df.index[movies_df['title'].str.lower() == titulo].tolist()
    if not idx:
        return "Película no encontrada"
    idx = idx[0]
    
    sim_scores = cosine_similarity(matriz_reducida[idx].reshape(1, -1), matriz_reducida)
    sim_scores = list(enumerate(sim_scores[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    movie_indices = [i[0] for i in sim_scores[1:num_recomendaciones+1]]
    
    return movies_df[['title', 'popularity']].iloc[movie_indices]

In [27]:
# Función para monitorear el uso de memoria del proceso
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

print(f"Uso de memoria: {get_memory_usage()} MB")

Uso de memoria: 749.140625 MB


In [28]:
# Guardar el modelo TF-IDF y la matriz reducida en archivos para futuras consultas
directory = os.path.join(RUTA_BASE, 'processed')
os.makedirs(directory, exist_ok=True)

joblib.dump(vectorizer, os.path.join(directory, 'vectorizer.pkl'))
joblib.dump(matriz_reducida, os.path.join(directory, 'matriz_reducida.pkl'))

print("✅ Modelos guardados correctamente.")

✅ Modelos guardados correctamente.


In [29]:
# Prueba de recomendación de películas
titulo_prueba = "stuart little"
print(f"Recomendaciones para '{titulo_prueba}':")
print(recomendar_peliculas(titulo_prueba))

Recomendaciones para 'stuart little':
                                                  title  popularity
4195                                    Stuart Little 2    9.274909
17324                                          Napoleon    0.863567
5398                                      Teacher's Pet    0.491334
18706  The Little Polar Bear: Lars and the Little Tiger    0.256353
17818                                     Hen, His Wife    0.003013
